# Plenary Protocols

In [1]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from pathlib import Path
import glob
import pandas as pd

In [3]:
VERBOSE = False

In [4]:
def show(table):
    with pd.option_context("display.max_colwidth", None):
        display(table)


def preview_lines(filepath, N=5):
    with open(filepath) as temp:
        head = [next(temp) for i in range(N)]
    temp.close()
    return head

### Data Loading
All csv-files in the specified directory will be parsed and concatenated.

In [5]:
path = "plpr-scraper/data/out"
all_files = glob.glob(path + "/*.csv")

df_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, escapechar="\\")
    df_list.append(df)

df = pd.concat(df_list, axis=0, ignore_index=True)
df_prior_shape = df.shape
show(df[:10])

,id,sitzung,wahlperiode,in_writing,speaker,speaker_cleaned,speaker_fp,sequence,text,filename,type,speaker_party
0,2490136,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,0,"Die Sitzung ist eröffnet. Nehmen Sie bitte Platz. - Liebe Kolleginnen und Kollegen, ich begrüße Sie zu unserer Plenarsitzung.\n\nDie Kollegin Dr. Lukrezia Jochimsen hat am 1. März ihren 75. Geburtstag gefeiert und die Kollegin Edelgard Bulmahn einige Tage später ihren 60. Geburtstag. Im Namen des gesamten Hauses möchte ich dazu auch auf diesem Wege noch einmal herzlich gratulieren und alle guten Wünsche übermitteln.",data/txt/17096.txt,chair,NaN
1,2490137,96,17,0,NaN,NaN,NaN,1,Beifall,data/txt/17096.txt,poi,NaN
2,2490138,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,2,Der Kollege Holger Haibach hat mit Wirkung vom 1. März auf seine Mitgliedschaft im Deutschen Bundestag verzichtet. Als seinen Nachfolger begrüße ich den Kollegen Helmut Heiderich.,data/txt/17096.txt,chair,NaN
3,2490139,96,17,0,NaN,NaN,NaN,3,Beifall,data/txt/17096.txt,poi,NaN
4,2490140,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,4,"Ebenso herzlich willkommen heiße ich den Kollegen Ingo Egloff, der als Nachfolger des Kollegen Olaf Scholz die Mitgliedschaft im Deutschen Bundestag erworben hat.",data/txt/17096.txt,chair,NaN
5,2490141,96,17,0,NaN,NaN,NaN,5,Beifall,data/txt/17096.txt,poi,NaN
6,2490142,96,17,0,Präsident Dr. Norbert Lammert,Dr. Norbert Lammert,norbert-lammert,6,"Die CDU/CSU-Fraktion hat mitgeteilt, dass die Kollegin Sibylle Pfeiffer ihr Amt als Schriftführerin niedergelegt hat.",data/txt/17096.txt,chair,NaN
7,2490143,96,17,0,Stefan Müller [Erlangen] [CDU/CSU],Stefan Müller,stefan-muller,7,Dem kann nicht zugestimmt werden!,data/txt/17096.txt,poi,cducsu
8,2490144,96,17,0,Sören Bartol [SPD],Sören Bartol,soren-bartol,8,Wollte sie keine Krawatte tragen?,data/txt/17096.txt,poi,spd
9,2490145,96,17,0,Dr. Dagmar Enkelmann [DIE LINKE],Dr. Dagmar Enkelmann,dagmar-enkelmann,9,Weil sie keine Krawatte trägt?,data/txt/17096.txt,poi,linke


### Data Cleansing
Quick look at the data before cleansing with Pandas-Profiling. We've got 592199 observations, many of which are not speeches but contributions by the *chair* (president and vice-presidents of the parliament) and *POIs*, points of information (interruptions or interjections by MPs other than the speaker or chair). Speeches span multiple rows and require concatenation.

In [6]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="Pre-Cleansing Pandas Profiling Report")

In [7]:
if VERBOSE:
    profile.to_notebook_iframe()

For now, we'll keep only speeches and replace newlines read-in literally.

In [8]:
df = df[~df["text"].isnull()].copy()
df.loc[:,"text"] = df.loc[:,"text"].str.replace("\n\n", " ").replace("\n", " ")
df.loc[:,"text"] = df.loc[:,"text"].astype(str)
df = df[df["type"] == "speech"]
df = df.reset_index(drop=True)
df.drop(["Unnamed: 0", "row.names"], axis=1, inplace=True, errors="ignore")
print(f"The shape is reduced from {df_prior_shape[0]} rows to {df.shape[0]}")
show(df[:3])

The shape is reduced from 592199 rows to 238648


,id,sitzung,wahlperiode,in_writing,speaker,speaker_cleaned,speaker_fp,sequence,text,filename,type,speaker_party
0,2490153,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,17,"Herr Präsident! Meine Damen und Herren! Am Freitag der letzten Woche, 14.45 Uhr Ortszeit, bebte in Japan die Erde. Seismologen maßen eine Stärke von 8,9, später korrigiert auf 9,0. Es war das schwerste Erdbeben in der Geschichte Japans. Sein Epizentrum lag circa 130 Kilometer östlich der Stadt Sendai und circa 400 Kilometer nordöstlich der japanischen Hauptstadt Tokio. Um 16 Uhr Ortszeit desselben Tages traf eine bis zu 10 Meter hohe Flutwelle auf die Ostküste der japanischen Hauptinsel Honshu. Sie richtete schwerste Verwüstungen an. Noch am Abend dieses Tages gab es Meldungen, wonach in einem Reaktor des Kernkraftwerks Fukushima I die Kühlung ausgefallen und im Atomkraftwerk Onagawa ein Feuer ausgebrochen war. Die japanische Regierung rief den atomaren Notstand aus. In den folgenden Tagen und Nächten erschütterten zahlreiche, zum Teil schwere Nachbeben das Land - und das bis heute. Erdbeben und Tsunami haben weite Landstriche von Japans Nordosten verwüstet. Ganze Ortschaften wurden ausgelöscht. Die Zahl der Opfer schnellt seit Tagen in die Höhe. Wie viele es tatsächlich sind - wir wissen es nicht. Zu viele Menschen werden vermisst. Unzählige Häuser und Straßen sind zerstört. Unendlich viele Menschen haben ihr Obdach verloren. Strom wird rationiert oder ist ganz weg. Treibstoff, Trinkwasser, Nahrungsmittel sind knapp. Rund um das Kernkraftwerk Fukushima wurde die Evakuierungszone seit Freitag immer wieder erweitert. Arbeiter dort führen einen ebenso - man kann es nicht anders sagen - heldenhaften wie verzweifelten Kampf gegen den atomaren Super-GAU. Sie setzen dabei nicht nur ihre Gesundheit aufs Spiel, sondern auch ihr Leben ein. Immer dramatischer entwickeln sich die Ereignisse dort: ausgefallene Kühlanlagen, Berichte über freiliegende Brennstäbe, die sich immer stärker erhitzen, Explosionen in verschiedenen Reaktoren, in einem Fall wohl auch mit der Folge der Beschädigung eines Sicherheitsbehälters, Radioaktivität tritt aus. Es ist davon auszugehen, dass es in drei der Anlagen zu schweren Schäden an den Reaktorkernen gekommen ist. Was uns angesichts all dieser Berichte und Bilder, die wir seit letztem Freitag sehen und zu verstehen versuchen, erfüllt, das sind Entsetzen, Fassungslosigkeit, Mitgefühl und Trauer. Die Katastrophe in Japan hat ein geradezu apokalyptisches Ausmaß, und es fehlen die Worte. Unsere tiefste Anteilnahme, unsere Gedanken und unsere Gebete sind bei den Menschen in Japan.",data/txt/17096.txt,speech,NaN
1,2490155,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,19,"In dieser Stunde schwerster Prüfung steht Deutschland an der Seite Japans. Was immer wir tun können, um den Menschen in Japan bei der Bewältigung dieser schier unfassbaren Katastrophe zu helfen, das werden wir weiter tun. Das habe ich Premierminister Kan übermittelt, und das hat auch der Bundesaußenminister seinem japanischen Kollegen gesagt. Experten des Technischen Hilfswerks haben in den vergangenen Tagen vor Ort bei der Suche nach Überlebenden geholfen. Ich danke ihnen, und ich danke den Helfern anderer Organisationen für ihren Einsatz für die Menschen in Japan.",data/txt/17096.txt,speech,NaN
2,2490157,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,21,"Ich danke allen Helfern des Krisenstabes im Auswärtigen Amt und der Botschaft vor Ort. Sie koordinieren unsere Hilfe. Sie unterstützen auch alle deutschen Staatsangehörigen im Krisengebiet bei einer Ausreise, wenn sie das wünschen. Auch die Vereinten Nationen haben ein Team nach Japan entsandt. Es soll die japanische Regierung dabei unterstützen, die Aufbaumaßnahmen zu koordinieren. Ebenfalls ihre Hilfe angeboten hat die Europäische Union. Liebe Kolleginnen und Kollegen, die Folgen dieser Katastrophe sind überhaupt noch nicht 

In [9]:
show(df[df["text"].str.contains("\. nan")][:5])

,id,sitzung,wahlperiode,in_writing,speaker,speaker_cleaned,speaker_fp,sequence,text,filename,type,speaker_party




### Option A: Slow but sizable junks without mid-sentence interruptions
It's slow but sizable junks without mid-sentence interruptions. Speeches span multiple rows, such cases can be joined partly, to finish on full sentences. A loop is computationally expensive (~25 min) but does the work!

In [10]:
%%time
path_plpr_a = "data/plpr_a.pkl"
if Path(path_plpr_a).exists():
    df_a = pd.read_pickle(path_plpr_a)

else:
    df_a = df.copy().reset_index(drop=True)
    for i in range(0, df_a.shape[0] - 1):
        if df_a.at[i, "speaker_fp"] == df_a.at[i + 1, "speaker_fp"]:
            # if type(df_a.at[i,'text']) == float:
            # print(df_a.at[i,'text'])
            if df_a.at[i, "text"].endswith("." or "!" or "?" or ":"):
                continue
            else:
                df_a.at[i + 1, "text"] = str(
                    df_a.at[i, "text"] + " " + df_a.at[i + 1, "text"]
                )
                df_a.drop(i, inplace=True)
            # stop the transformation as the loop runs out of index

    df_a = df_a.reset_index(drop=True)  # reindexing
    df_a.to_pickle(path_plpr_a)
show(df_a[:3])

,id,sitzung,wahlperiode,in_writing,speaker,speaker_cleaned,speaker_fp,sequence,text,filename,type,speaker_party
0,2490153,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,17,"Herr Präsident! Meine Damen und Herren! Am Freitag der letzten Woche, 14.45 Uhr Ortszeit, bebte in Japan die Erde. Seismologen maßen eine Stärke von 8,9, später korrigiert auf 9,0. Es war das schwerste Erdbeben in der Geschichte Japans. Sein Epizentrum lag circa 130 Kilometer östlich der Stadt Sendai und circa 400 Kilometer nordöstlich der japanischen Hauptstadt Tokio. Um 16 Uhr Ortszeit desselben Tages traf eine bis zu 10 Meter hohe Flutwelle auf die Ostküste der japanischen Hauptinsel Honshu. Sie richtete schwerste Verwüstungen an. Noch am Abend dieses Tages gab es Meldungen, wonach in einem Reaktor des Kernkraftwerks Fukushima I die Kühlung ausgefallen und im Atomkraftwerk Onagawa ein Feuer ausgebrochen war. Die japanische Regierung rief den atomaren Notstand aus. In den folgenden Tagen und Nächten erschütterten zahlreiche, zum Teil schwere Nachbeben das Land - und das bis heute. Erdbeben und Tsunami haben weite Landstriche von Japans Nordosten verwüstet. Ganze Ortschaften wurden ausgelöscht. Die Zahl der Opfer schnellt seit Tagen in die Höhe. Wie viele es tatsächlich sind - wir wissen es nicht. Zu viele Menschen werden vermisst. Unzählige Häuser und Straßen sind zerstört. Unendlich viele Menschen haben ihr Obdach verloren. Strom wird rationiert oder ist ganz weg. Treibstoff, Trinkwasser, Nahrungsmittel sind knapp. Rund um das Kernkraftwerk Fukushima wurde die Evakuierungszone seit Freitag immer wieder erweitert. Arbeiter dort führen einen ebenso - man kann es nicht anders sagen - heldenhaften wie verzweifelten Kampf gegen den atomaren Super-GAU. Sie setzen dabei nicht nur ihre Gesundheit aufs Spiel, sondern auch ihr Leben ein. Immer dramatischer entwickeln sich die Ereignisse dort: ausgefallene Kühlanlagen, Berichte über freiliegende Brennstäbe, die sich immer stärker erhitzen, Explosionen in verschiedenen Reaktoren, in einem Fall wohl auch mit der Folge der Beschädigung eines Sicherheitsbehälters, Radioaktivität tritt aus. Es ist davon auszugehen, dass es in drei der Anlagen zu schweren Schäden an den Reaktorkernen gekommen ist. Was uns angesichts all dieser Berichte und Bilder, die wir seit letztem Freitag sehen und zu verstehen versuchen, erfüllt, das sind Entsetzen, Fassungslosigkeit, Mitgefühl und Trauer. Die Katastrophe in Japan hat ein geradezu apokalyptisches Ausmaß, und es fehlen die Worte. Unsere tiefste Anteilnahme, unsere Gedanken und unsere Gebete sind bei den Menschen in Japan.",data/txt/17096.txt,speech,NaN
1,2490155,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,19,"In dieser Stunde schwerster Prüfung steht Deutschland an der Seite Japans. Was immer wir tun können, um den Menschen in Japan bei der Bewältigung dieser schier unfassbaren Katastrophe zu helfen, das werden wir weiter tun. Das habe ich Premierminister Kan übermittelt, und das hat auch der Bundesaußenminister seinem japanischen Kollegen gesagt. Experten des Technischen Hilfswerks haben in den vergangenen Tagen vor Ort bei der Suche nach Überlebenden geholfen. Ich danke ihnen, und ich danke den Helfern anderer Organisationen für ihren Einsatz für die Menschen in Japan.",data/txt/17096.txt,speech,NaN
2,2490157,96,17,0,"Dr. Angela Merkel, Bundeskanzlerin","Dr. Angela Merkel,",angela-merkel,21,"Ich danke allen Helfern des Krisenstabes im Auswärtigen Amt und der Botschaft vor Ort. Sie koordinieren unsere Hilfe. Sie unterstützen auch alle deutschen Staatsangehörigen im Krisengebiet bei einer Ausreise, wenn sie das wünschen. Auch die Vereinten Nationen haben ein Team nach Japan entsandt. Es soll die japanische Regierung dabei unterstützen, die Aufbaumaßnahmen zu koordinieren. Ebenfalls ihre Hilfe angeboten hat die Europäische Union. Liebe Kolleginnen und Kollegen, die Folgen dieser Katastrophe sind überhaupt noch nicht 

CPU times: user 228 ms, sys: 86.6 ms, total: 314 ms
Wall time: 314 ms


### Option B: Fast per speech with some column detail disregarded


In [11]:
for name in df.columns:
    print(df.at[0, name] == df.at[1, name], name)

False id
True sitzung
True wahlperiode
True in_writing
True speaker
True speaker_cleaned
True speaker_fp
False sequence
False text
True filename
True type
False speaker_party


In [12]:
%%time
if 1 == 1:
    df_b = df.copy()
    df_b["speaker_fp_duplicate"] = df_b["speaker_fp"].copy()
    adjacent_rows_grouper = (df_b["speaker_fp"].shift() != df_b["speaker_fp"]).cumsum()
    df_b["id"] = df_b["id"].astype(str)
    df_b = (
        df_b.groupby(
            [
                adjacent_rows_grouper,
                "sitzung",
                "wahlperiode",
                "speaker_cleaned",
                "filename",
                "type",
            ]
        )["text"]
        .apply(" ".join)
        .reset_index()
    )
    df_b.to_pickle("data/plpr_b.pkl")
    show(df_b[:5])

speaker_fp  sitzung  wahlperiode     speaker_cleaned            filename  \
0           1       96           17  Dr. Angela Merkel,  data/txt/17096.txt   
1           2       96           17      Sigmar Gabriel  data/txt/17096.txt   
2           3       96           17    Birgit Homburger  data/txt/17096.txt   
3           4       96           17     Dr. Gregor Gysi  data/txt/17096.txt   
4           5       96           17       Volker Kauder  data/txt/17096.txt   

     type  \
0  speech   
1  speech   
2  speech   
3  speech   
4  speech   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

CPU times: user 1.67 s, sys: 231 ms, total: 1.9 s
Wall time: 1.91 s


### All Text
To model topics, metainformation for speeches is not relevant. Everything in the text column can be glued together for that purpose.

In [13]:
# Option A
path_alltext_a = Path("data/plpr_alltext_a.txt")

df_a["text"].to_csv(path_alltext_a, sep=" ", index=False, header=False)
preview_lines(path_alltext_a, N=2)

['"Herr Präsident! Meine Damen und Herren! Am Freitag der letzten Woche, 14.45 Uhr Ortszeit, bebte in Japan die Erde. Seismologen maßen eine Stärke von 8,9, später korrigiert auf 9,0. Es war das schwerste Erdbeben in der Geschichte Japans. Sein Epizentrum lag circa 130 Kilometer östlich der Stadt Sendai und circa 400 Kilometer nordöstlich der japanischen Hauptstadt Tokio. Um 16 Uhr Ortszeit desselben Tages traf eine bis zu 10 Meter hohe Flutwelle auf die Ostküste der japanischen Hauptinsel Honshu. Sie richtete schwerste Verwüstungen an. Noch am Abend dieses Tages gab es Meldungen, wonach in einem Reaktor des Kernkraftwerks Fukushima I die Kühlung ausgefallen und im Atomkraftwerk Onagawa ein Feuer ausgebrochen war. Die japanische Regierung rief den atomaren Notstand aus. In den folgenden Tagen und Nächten erschütterten zahlreiche, zum Teil schwere Nachbeben das Land - und das bis heute. Erdbeben und Tsunami haben weite Landstriche von Japans Nordosten verwüstet. Ganze Ortschaften wurden

In [14]:
# Option B
path_alltext_b = Path("data/plpr_alltext_b.txt")

# df_b["text"] = df_b["text"].str.replace(". nan", ".")
# df_b["text"][df_b["text"] != "nan"].to_csv(path_alltext_b, sep=" ", index=False, header=False)
df_b["text"].to_csv(path_alltext_b, sep=" ", index=False, header=False)